In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from cts_recommender import logging_setup
logging_setup.setup_logging('INFO')

In [19]:
from cts_recommender.preprocessing import ML_feature_processing
from cts_recommender.settings import get_settings

cfg = get_settings()

processed_df, enriched_df = ML_feature_processing.load_processed_and_enriched_programming(
    data_path_processed=cfg.processed_dir / "programming.parquet", 
    data_path_enriched=cfg.processed_dir / "programming_enriched.parquet"
    )

ML_df = ML_feature_processing.build_X_features(processed_df, enriched_df)


In [14]:
import pandas as pd
from cts_recommender.features.schemas import TARGET_FEATURE
from sklearn.model_selection import train_test_split

X = pd.get_dummies(ML_df.drop(columns=[TARGET_FEATURE]))
y = ML_df[TARGET_FEATURE]

# Random split of data movies and non-movies together
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [16]:
len(X_train.columns)

67

In [24]:
from cts_recommender.models.audience_ratings_regressor import AudienceRatingsRegressor

model = AudienceRatingsRegressor()
model.prepare_data(ML_features_path=cfg.processed_dir / "ML_features.parquet")

2025-10-01 14:44:30 INFO cts_recommender.models.audience_ratings_regressor: Loading processed and enriched programming data...
2025-10-01 14:44:30 INFO cts_recommender.models.audience_ratings_regressor: Building feature matrix and targets...
2025-10-01 14:44:30 INFO cts_recommender.models.audience_ratings_regressor: Encoding categorical variables...
2025-10-01 14:44:30 INFO cts_recommender.models.audience_ratings_regressor: Feature matrix shape before encoding: (85831, 35)
2025-10-01 14:44:30 INFO cts_recommender.models.audience_ratings_regressor: Feature matrix shape after encoding: (85831, 67)
2025-10-01 14:44:30 INFO cts_recommender.models.audience_ratings_regressor: Target distribution: mean=14.811, std=28.620
2025-10-01 14:44:30 INFO cts_recommender.models.audience_ratings_regressor: Training set: 68664 samples
2025-10-01 14:44:30 INFO cts_recommender.models.audience_ratings_regressor: Test set: 17167 samples


X.columns before encoding Index(['hour', 'weekday', 'is_weekend', 'duration_min', 'season',
       'public_holiday', 'channel', 'original_language', 'popularity',
       'revenue', 'vote_average', 'adult', 'missing_release_date',
       'missing_tmdb', 'is_movie', 'genre_Action', 'genre_Animation',
       'genre_Aventure', 'genre_Comédie', 'genre_Crime', 'genre_Documentaire',
       'genre_Drame', 'genre_Familial', 'genre_Fantastique', 'genre_Guerre',
       'genre_Histoire', 'genre_Horreur', 'genre_Musique', 'genre_Mystère',
       'genre_Romance', 'genre_Science-Fiction', 'genre_Thriller',
       'genre_Téléfilm', 'genre_Western', 'movie_age'],
      dtype='object')


(        hour  weekday  is_weekend  duration_min  public_holiday  popularity  \
 184261    12        1           0     40.383333               0         0.0   
 51523     24        6           1     50.500000               1         0.0   
 281251    14        0           0     66.350000               1         0.0   
 379183     7        2           0     11.000000               0         0.0   
 248689    17        0           0     41.150000               1         0.0   
 ...      ...      ...         ...           ...             ...         ...   
 38359     25        2           0     20.500000               0         0.0   
 334723    19        6           1     32.000000               0         0.0   
 469489    20        2           0      2.366667               0         0.0   
 5431       7        4           0     66.416667               1         0.0   
 96439     10        6           1     30.000000               0         0.0   
 
         revenue  vote_average  adult 

In [25]:
from cts_recommender.pipelines.audience_ratings_regression_pipeline import train_audience_ratings_model

audience_ratings_model, train_metrics, test_metrics = train_audience_ratings_model(ML_features_path = cfg.processed_dir / "ML_features.parquet")


2025-10-01 14:44:37 INFO cts_recommender.utils.reproducibility: Setting global random seed to 42
2025-10-01 14:44:37 INFO cts_recommender.utils.reproducibility: Global random seed set successfully
2025-10-01 14:44:37 INFO cts_recommender.pipelines.audience_ratings_regression_pipeline: Using random seed: 42
2025-10-01 14:44:37 INFO cts_recommender.models.audience_ratings_regressor: Loading processed and enriched programming data...


2025-10-01 14:44:37 INFO cts_recommender.models.audience_ratings_regressor: Building feature matrix and targets...
2025-10-01 14:44:37 INFO cts_recommender.models.audience_ratings_regressor: Encoding categorical variables...
2025-10-01 14:44:37 INFO cts_recommender.models.audience_ratings_regressor: Feature matrix shape before encoding: (85831, 35)
2025-10-01 14:44:37 INFO cts_recommender.models.audience_ratings_regressor: Feature matrix shape after encoding: (85831, 67)
2025-10-01 14:44:37 INFO cts_recommender.models.audience_ratings_regressor: Target distribution: mean=14.811, std=28.620
2025-10-01 14:44:37 INFO cts_recommender.models.audience_ratings_regressor: Training set: 68664 samples
2025-10-01 14:44:37 INFO cts_recommender.models.audience_ratings_regressor: Test set: 17167 samples
2025-10-01 14:44:37 INFO cts_recommender.models.audience_ratings_regressor: Training Random Forest regressor...
2025-10-01 14:45:08 INFO cts_recommender.models.training: MSE RandomForestRegressor: 13